In [1]:
import os
import tempfile
import shutil
import numpy as np
import scipy as sp
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

from sklearn.metrics import mean_squared_error


class GeographicalModel:

    def __init__(self,
        algorithm: str, 
        parameters: dict,
        bandwidth_type: str,
        kernel: str,
        kernel_param: list):

        self.algorithm = algorithm
        self.parameters = parameters
        self.bandwidth_type = bandwidth_type
        self.kernel = kernel
        self.kernel_param = kernel_param

    def kernel_gaussian(self, distances, b=1): 

        return np.exp(-0.5*(distances/b)**2)
    
    def kernel_none(self, distances):

        return distances
    
    def interpolate_linear(self, coordinates, values, new_coordinates):
         
        # Create an empty array to store the interpolated values
        interpolated_values = np.zeros(len(new_coordinates))

        for i, new_coord in enumerate(new_coordinates):
            # Get the new x and y coordinates
            x_new, y_new = new_coord
        
            # Find the four closest points surrounding the new coordinates
            distances = np.sqrt((coordinates[:, 0] - x_new)**2 + (coordinates[:, 1] - y_new)**2)
            indices = np.argsort(distances)[:4]
            closest_coords = coordinates[indices]
            closest_values = values[indices]
        
            # Compute the weights based on inverse distance
            weights = 1 / distances[indices]
            weights /= weights.sum()
        
            # Interpolate the value for the new coordinates
            interpolated_values[i] = np.dot(weights, closest_values)

        return interpolated_values
    
    import numpy as np

    def create_stepped_matrix(self, shape, step):

        rows, cols = shape
        # Initialize the matrix with zeros
        matrix = np.zeros((rows, cols), dtype=int)

        # Assign values to the matrix in steps
        for r in range(0, rows, step):
            for c in range(0, cols, step):

                # Calculate the value based on the row and column indices
                value = (r // step) * (cols // step) + (c // step)
                # Assign the value to the current window
                matrix[r:r+step, c:c+step] = value
        return matrix    

    def tune(
        self,
        coordinates_train,
        features_train,
        labels_train,
        n_jobs: int,
        bandwidths: list,
        step=1,
        limits=False,
        limits_ind=[0,0,0,0]):

        self.limits = limits
        self.limits_ind = limits_ind

        # add column containg order 
        coordinates_train = np.hstack((coordinates_train, np.arange(0, coordinates_train.shape[0], 1).reshape((-1,1))))

        # split the data according to limits
        if limits == True:

            limits_indx = np.where((coordinates_train[:,0] > limits_ind[0]) & (coordinates_train[:,0] < limits_ind[1])
                                    & (coordinates_train[:,1] > limits_ind[2]) & (coordinates_train[:,1] < limits_ind[3]))[0]
            coordinates_train_limits = coordinates_train[limits_indx]
        else:
            coordinates_train_limits=coordinates_train

        # create an instance of scikit-learn estimator
        Model = eval(self.algorithm)(**self.parameters)

        min_x, max_x = limits_ind[0], limits_ind[1]
        min_y, max_y = limits_ind[2], limits_ind[3]

        mask_matrix = self.create_stepped_matrix((max_x-min_x, max_y-min_y), step)

        x_coord_linspace = np.arange(limits_ind[0], limits_ind[1], 1)
        y_coord_linspace = np.arange(limits_ind[2], limits_ind[3], 1)

        bandwidth_results = np.zeros((coordinates_train.shape[0], len(bandwidths)))

        x_grid, y_grid = np.meshgrid(x_coord_linspace, y_coord_linspace)
        
        # --------CYCLE-----------------------
        def delayed_function(region_id,
                             bandwidth_value,
                             kernel,
                             kernel_param,
                             coordinates_train,
                             features_train, 
                             label_train
                            
        ):
            
            x_coord_centroid = x_grid[np.where(mask_matrix == region_id, True, False)].mean()
            y_coord_centroid = y_grid[np.where(mask_matrix == region_id, True, False)].mean()

            # calculate distance matrix for prediction samples
            distance_matrix = np.sqrt((x_coord_centroid - coordinates_train[:, 0])**2 + (y_coord_centroid - coordinates_train[:, 1])**2)

            # get indices for inner
            inner_boolean_matrix = np.where(distance_matrix < step/2, True, False)

            if inner_boolean_matrix.sum() > 0:

                outer_boolean_matrix = np.where(distance_matrix < bandwidth_value, True, False) ^ inner_boolean_matrix

                kernel_matrix =  getattr(self, f'kernel_{kernel}')(distance_matrix.flatten()[outer_boolean_matrix], *kernel_param)

                Model.fit(features_train[outer_boolean_matrix], label_train[outer_boolean_matrix])
                #Model.fit(features_train[outer_boolean_matrix], label_train[outer_boolean_matrix], kernel_matrix)
                prediction = Model.predict(features_train[inner_boolean_matrix])

                error = mean_squared_error(label_train[inner_boolean_matrix], prediction, squared=False)
                error_matrix[inner_boolean_matrix] = error
            
            else:
                error_matrix[inner_boolean_matrix] = -1

            return error_matrix

        for n_bandwidth, bandwidth_value in tqdm(enumerate(bandwidths), total=len(bandwidths), position=0):


            path = tempfile.mkdtemp()
            temp_path = os.path.join(path,'result_array.mmap')

            error_matrix = np.memmap(temp_path, dtype=float, shape=(coordinates_train.shape[0]), mode='w+')

            error_matrix = Parallel(n_jobs=n_jobs)(delayed(delayed_function) (
                region_id, 
                bandwidth_value, 
                self.kernel, 
                self.kernel_param, 
                coordinates_train, 
                features_train, 
                labels_train) 

                for region_id in (np.unique(mask_matrix)))
            
            bandwidth_results[:, n_bandwidth] = error_matrix[0]
            
            try:
                shutil.rmtree(path)
            except:
                pass
            

        single_bandwidth_error = np.mean(bandwidth_results, axis=0)
        single_best_bandwidth = bandwidths[np.argmin(single_bandwidth_error)]

        best_bw_ind = (np.argmin(bandwidth_results, axis=1))
        best_bw = np.array(list(map(lambda x: bandwidths[x] , best_bw_ind)))

        self.best_bw = best_bw
        self.single_best_bandwidth = single_best_bandwidth

        return single_best_bandwidth, best_bw

    def predict(self,
                bandwidth,
                bandwidth_type,
                interpolation,
                coordinates_train,
                features_train,
                labels_train,
                coordinates_test,
                features_test,
                n_jobs):
        

        coordinates_test = np.hstack((coordinates_test, np.arange(0, coordinates_test.shape[0], 1).reshape((-1,1))))

        if isinstance(bandwidth, int):
            bandwidth_values = np.full(shape=(coordinates_test.shape[0]), fill_value=bandwidth)
        else:
            interpolation_str = f'interpolate_{interpolation}'
            bandwidth_values = getattr(self, interpolation_str)(coordinates_train, bandwidth, coordinates_test[:, :2])

        Model = eval(self.algorithm)(**self.parameters)

        path = tempfile.mkdtemp()
        temp_path = os.path.join(path,'prediction_array.mmap')

        prediction_matrix = np.memmap(temp_path, dtype=float, shape=(coordinates_test.shape[0]), mode='w+')

        def delayed_function(location, 
                             bandwidth_values,  
                             coordinates_train, 
                             features_train, 
                             labels_train,
                             features_test
                             ):
            

            order_n = location[2]

            distance_matrix = np.sqrt(((location[0] - coordinates_train[:, 0])**2 + (location[1] - coordinates_train[:, 1])**2))

            if self.bandwidth_type == 'fixed':

                boolean_matrix = np.where(distance_matrix <= bandwidth_values[int(order_n)], True, False).flatten()
                
            elif self.bandwidth_type == 'adaptive':
                    
                boolean_matrix = np.full(shape=distance_matrix.shape, fill_value=False)
                n_closest_ind =  np.argsort(distance_matrix)[:bandwidth_values+1]
                boolean_matrix[n_closest_ind] = True

            boolean_matrix[int(order_n)] = False

            weight_matrix = np.zeros((distance_matrix.shape))
            kernel_matrix =  getattr(self, f'kernel_{self.kernel}')(distance_matrix.flatten()[boolean_matrix], *self.kernel_param)
            weight_matrix[boolean_matrix] = kernel_matrix
            
            #Model.fit(features_train[boolean_matrix], labels_train[boolean_matrix], kernel_matrix)
            Model.fit(features_train[boolean_matrix], labels_train[boolean_matrix])
            prediction = Model.predict(features_test[int(order_n), :].reshape(1, -1))
            prediction_matrix[int(order_n)] = prediction

            return prediction_matrix
        

        prediction_matrix = Parallel(n_jobs=n_jobs)(delayed(delayed_function) (
                location, 
                bandwidth_values,   
                coordinates_train, 
                features_train, 
                labels_train,
                features_test)

            for location in coordinates_test)
        
        prediction_arr = np.array(prediction_matrix[0].tolist())

        try:
            shutil.rmtree(path)
        except:
            pass

        return prediction_arr